In [1]:
%pip install elasticsearch

  Obtaining dependency information for elasticsearch from https://files.pythonhosted.org/packages/ab/dc/87c47f9c42f8188d24f6c589ed0e8168a0fef7091e5ccffdbc950be7ab66/elasticsearch-8.15.0-py3-none-any.whl.metadata
  Using cached elasticsearch-8.15.0-py3-none-any.whl.metadata (8.7 kB)
  Obtaining dependency information for elastic-transport<9,>=8.13 from https://files.pythonhosted.org/packages/1d/4a/df755b094170351b8199074fe8d0dfe6d639ab78a2e8893b556620ae982a/elastic_transport-8.15.0-py3-none-any.whl.metadata
  Using cached elastic_transport-8.15.0-py3-none-any.whl.metadata (3.6 kB)
Using cached elasticsearch-8.15.0-py3-none-any.whl (523 kB)
Using cached elastic_transport-8.15.0-py3-none-any.whl (64 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
from elasticsearch import Elasticsearch

client = Elasticsearch(
    "http://elasticsearch:9200",
)

documents = [
    {"index": {"_index": "test", "_id": "9780441017225"}},
    {
        "name": "Revelation Space",
        "author": "Alastair Reynolds",
        "release_date": "2000-03-15",
        "page_count": 585,
        "_extract_binary_content": True,
        "_reduce_whitespace": True,
        "_run_ml_inference": True,
    },
    {"index": {"_index": "test", "_id": "9780451524935"}},
    {
        "name": "1984",
        "author": "George Orwell",
        "release_date": "1985-06-01",
        "page_count": 328,
        "_extract_binary_content": True,
        "_reduce_whitespace": True,
        "_run_ml_inference": True,
    },
    {"index": {"_index": "test", "_id": "9781451673319"}},
    {
        "name": "Fahrenheit 451",
        "author": "Ray Bradbury",
        "release_date": "1953-10-15",
        "page_count": 227,
        "_extract_binary_content": True,
        "_reduce_whitespace": True,
        "_run_ml_inference": True,
    },
    {"index": {"_index": "test", "_id": "9780060850524"}},
    {
        "name": "Brave New World",
        "author": "Aldous Huxley",
        "release_date": "1932-06-01",
        "page_count": 268,
        "_extract_binary_content": True,
        "_reduce_whitespace": True,
        "_run_ml_inference": True,
    },
    {"index": {"_index": "test", "_id": "9780385490818"}},
    {
        "name": "The Handmaid's Tale",
        "author": "Margaret Atwood",
        "release_date": "1985-06-01",
        "page_count": 311,
        "_extract_binary_content": True,
        "_reduce_whitespace": True,
        "_run_ml_inference": True,
    },
]

print(client.bulk(operations=documents, pipeline="ent-search-generic-ingestion"))

print(client.search(index="test", q="snow"))


{'errors': False, 'took': 0, 'ingest_took': 0, 'items': [{'index': {'_index': 'test', '_id': '9780441017225', '_version': 3, 'result': 'updated', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 16, '_primary_term': 13, 'status': 200}}, {'index': {'_index': 'test', '_id': '9780451524935', '_version': 3, 'result': 'updated', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 17, '_primary_term': 13, 'status': 200}}, {'index': {'_index': 'test', '_id': '9781451673319', '_version': 3, 'result': 'updated', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 18, '_primary_term': 13, 'status': 200}}, {'index': {'_index': 'test', '_id': '9780060850524', '_version': 3, 'result': 'updated', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 19, '_primary_term': 13, 'status': 200}}, {'index': {'_index': 'test', '_id': '9780385490818', '_version': 3, 'result': 'updated', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 

In [2]:
import pyspark
from delta import *

builder = pyspark.sql.SparkSession.builder.appName("LocalDelta") \
    .master("local") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.4.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.extensions", "org.elasticsearch:elasticsearch-spark-30_2.12:8.15.1") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [8]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
data2 = [("James","","Smith","36636","M",3000),
    ("Michael","Rose","","40288","M",4000),
    ("Robert","","Williams","42114","M",4000),
    ("Maria","Anne","Jones","39192","F",4000),
    ("Jen","Mary","Brown","1234","F",-1)
  ]

schema = StructType([ \
    StructField("firstname",StringType(),True), \
    StructField("middlename",StringType(),True), \
    StructField("lastname",StringType(),True), \
    StructField("id", StringType(), True), \
    StructField("gender", StringType(), True), \
    StructField("salary", IntegerType(), True) \
  ])
 
df = spark.createDataFrame(data=data2,schema=schema)
df.write.format("delta").option("mergeSchema", "true").mode("overwrite").save("local_lake/scratch/users")

In [9]:
# Reading delta table
df = spark.read.format("delta").load("local_lake/scratch/users")

In [5]:
df.show()

+---------+----------+--------+-----+------+------+
|firstname|middlename|lastname|   id|gender|salary|
+---------+----------+--------+-----+------+------+
|    James|          |   Smith|36636|     M|  3000|
|  Michael|      Rose|        |40288|     M|  4000|
|   Robert|          |Williams|42114|     M|  4000|
|    Maria|      Anne|   Jones|39192|     F|  4000|
|      Jen|      Mary|   Brown| 1234|     F|    -1|
+---------+----------+--------+-----+------+------+



In [7]:
df.write.format("org.elasticsearch.spark.sql")\
    .option("es.nodes", "http://elasticsearch:9200")\
    .option("es.nodes.discovery", "false")\
    .option("es.nodes.wan.only", "true")\
    .option("es.index.auto.create", "true")\
    .option("es.mapping.id", "id")\
    .option("es.mapping.exclude", "id")\
    .mode("append") \
    .save("test")